# Notebook Prep

In [ ]:
#import libraries

import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import scipy

In [ ]:
#Define variable: 'path'

path = r'C:\Users\PC Planet\Desktop\Self-Education\Data Immersion\Achievement 4\Instacart Basket Analysis'

In [ ]:
#Import test dataframe

df = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'dataframe_after_exclusion.pkl'))

In [ ]:
#Create subset of test dataframe

df2 = df[:1000]

# Test Code

In [ ]:
#Sort dataframe by user_id and order_number

df2 = df2.sort_values(['user_id','order_number'])

In [ ]:
#Add same_user flag

df2['same_user'] = np.where(df2['user_id'].eq(df2['user_id'].shift()), True, False)

In [ ]:
#Reset index

df2.reset_index(inplace = True)

In [ ]:
apv = []

In [ ]:
profile = []

In [ ]:
#Nested For-Loops to Add Strings to 'profile' list

for index, row in df2.iterrows():

    if row['same_user'] == 'False':
        profile.clear()

#Income conditional

        income = row['income']
    
        if income < 32500:
            profile.append('Struggling')
        elif (income >= 32500) & (income < 65000):
            profile.append('Lower-Middle Class')
        elif (income >= 65000) & (income < 97500):
            profile.append('Upper-Middle Class')
        elif (income >= 97500) & (income < 195000):
            profile.append('High-Class')
        elif (income >= 195000):
            profile.append('Wealthy')
        else: pass

#Age conditional

        age = row['age']
    
        if age < 30:
            profile.append('Young')
        elif (age >= 30) & (age < 45):
            profile.append('Adult')
        elif (age >= 45) & (age < 65):
            profile.append('Middle-Aged')
        elif age >= 65:
            profile.append('Elderly')
        else: pass

#Alcohol conditional and loop

    #Create subset based on current 'user_id'
        
        df_user = df2[df2['user_id']==row['user_id']]
        
    #Create second subset based on first subset and reduce to unique rows
        
        df_time = df_user[['order_number', 'days_since_prior_order']]
        df_time.drop_duplicates(inplace = True)
        df_time.reset_index(inplace = True)
            
    #Define initial variables
            
        last_order = df_time['order_number'].max()
        day_limit = df_time.loc[df_time['order_number'] == last_order, 'days_since_prior_order']
        earliest_order = last_order
        number_of_orders = 1
                    
    #While loop to count orders over last 150 days and iterate variables
                
        while all((day_limit <= 150) & (earliest_order > 1)):
            df_time.drop(df_time.tail(1).index, inplace = True)       
            earliest_order = df_time['order_number'].max()
            day_limit = day_limit + df_time.loc[df_time['order_number'] == earliest_order, 'days_since_prior_order']
            number_of_orders = number_of_orders + 1
            
    #Drop rows outside of 150-day range 
        
        df_user.drop(df_user[df_user['order_number'] < earliest_order].index, inplace = True)
        df_user.reset_index(inplace = True)
        
    #Determine amount of alcohol purchased
        
        if '5' in df_user['department_id']:
            alcohol_purchased = df_user['department_id'].value_counts()['5']
        else: alcohol_purchased = 0
            
    #Calculate mean alcohol purchases per order
        
        alcohol_mean = alcohol_purchased / number_of_orders 
            
    #Determine appropriate appendage
            
        if alcohol_purchased == 0:
            profile.append('Abstinent')
        elif alcohol_mean < 3:
            profile.append('Casual-Drinking')
        elif alcohol_mean >= 3:
            profile.append('Heavy-Drinking')

#Diet Loop
    
    #Determine amount of produce purchased
        
        if '4' in df_user['department_id']:
            produce = df_user['department_id'].value_counts()['4']
        else: produce = 0 
        
    #Determine amount of dairy purchased
        
        if '16' in df_user['department_id']:
            dairy = df_user['department_id'].value_counts()['16']
        else: dairy = 0
            
    #Determine amount of meat purchased
        
        if '12' in df_user['department_id']:
            meat = df_user['department_id'].value_counts()['12']
        else: meat = 0
        
    #Meat mean
        
        meat_mean = meat / number_of_orders
                
    #Determine appropriate appendage
        
        if (produce > 0) & (dairy == 0) & (meat == 0):
            profil.append('Vegan')
        elif (produce > 0) & (meat == 0):
            profile.append('Vegetarian')
        elif meat_mean > 5:
            profile.append('Carnivorous')
        else: pass

#Marital Status conditional

        if row['marital_status']=='married':
            profile.append('Married')
        elif row['marital_status']=='divorced/widowed':
            profile.append('Divorced/Widowed')
        else: profile.append('Single')

#Gender conditional
    
        if row['gender']=='male':
            profile.append('Male')
        else: profile.append('Female')

#Dependants Loop

        dependants = row['dependants']
    
        if '18' in df_user['department_id']:
            babies = df_user['department_id'].value_counts()['18']
        else: babies = 0
            
        babies_mean = babies / number_of_orders
    
        if dependants == 0:
            dependants_append = ''
        elif (dependants > 0) & (babies_mean >= 1):
            dependants_append = 'with Infant Children'
        else: dependants_append = 'with Children'
            
        profile.append(dependants_append)

#Pets
    
    #Determine amount of pet products purchased
        
        if '8' in df_user['department_id']:
            pets = df_user['department_id'].value_counts()['8']
        else: pets = 0
        
    #Append pets profile
        
        if pets >= 2:
            if dependants_append == '':
                profile.append('with Pets')
            else: profile.append('and Pets')
            
        else: pass
              
#Append 'profile' list to 'apv' list
        
        apv.append(profile)
        prev_profile = profile
    
#Append old profile if same user
    
    else: apv.append(prev_profile)
        


In [ ]:
df2['assumed_profile'] = apv

In [ ]:
df2['assumed_profile'].value_counts()

In [ ]:
df2.head(75)

# Check Success